In [ ]:
import numpy as np
import torch

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from mlp import MLP

In [ ]:
features = np.load('./data/lbp/features.npy') # [num, 256]
labels = np.load('./data/lbp/labels.npy') # [num, 4]

In [ ]:
# Move the model to GPU if available
# device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
device = 'mps'
print("Device is set to:", device)

features = torch.tensor(features, dtype=torch.float, device=device)
labels = torch.tensor(labels, dtype=torch.float, device=device)

# Print the data types
print(f'Features dtype: {features.dtype}')
print(f'Features shape: {features.shape}')
print(f'Labels dtype: {labels.dtype}')
print(f'Labels shape: {labels.shape}')

# Create a TensorDataset
dataset = TensorDataset(features, labels)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=128, shuffle=True) # Stratify?

In [ ]:
# Example parameters
input_dim = features.shape[1]  # This should be equal to num_bins (self.dec_values)
hidden_dims = [256, 128]
output_dim = 4  # Four classes
dropout_rate = 0.3

model = MLP(input_dim, hidden_dims, output_dim, dropout_rate)

model.to(device)

In [ ]:
import torch.nn as nn
import torch.optim as optim

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 1000

avg_loss = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for features, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    avg_loss = running_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')
    
torch.save(model, f"./models/mlp_model_{avg_loss:.4f}.pth")